In [1]:
import pandas as pd
import numpy as np
import cv2
import os
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, Activation
from matplotlib import pyplot as plt

In [2]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperas import optim

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam, SGD

In [9]:
def get_train_test_data():
    # Create dictionary for alphabets and related numbers
    alphabets_dic = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J',
                 10: 'K', 11: 'L', 12: 'M', 13: 'N', 14: 'O', 15: 'P', 16: 'Q', 17: 'R', 18: 'S', 19: 'T',
                 20: 'U', 21: 'V', 22: 'W', 23: 'X', 24: 'Y', 25: 'Z', 26: '0', 27: '1', 28: '2', 29:'3',
                 30: '4', 31: '5', 32: '6', 33: '7', 34: '8', 35: '9'}

    alphabets = ['0','1','2','3','4','5','6','7','8','9','A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']
    dataset_classes = []

    for cls in alphabets:
        dataset_classes.append([cls])

    # Load dataset
    d = open("./data/alternate_data.pickle","rb")
    l = open("./data/alternate_data_labels.pickle","rb")
    data = pickle.load(d)
    labels = pickle.load(l)
    label_list = []
    for l in labels:
        label_list.append([l])

    # One hot encoding format for output
    ohe = OneHotEncoder(handle_unknown='ignore', categorical_features=None)
    ohe.fit(dataset_classes)
    labels_ohe = ohe.transform(label_list).toarray()

    data = np.array(data)
    labels = np.array(labels)

    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(data, labels_ohe, test_size=0.20, random_state=42)
    print(X_train.shape)
    print(X_test.shape)
    print(y_train.shape)
    print(y_test.shape)

    X_train = X_train.reshape(X_train.shape[0],28,28,1)
    X_test = X_test.reshape(X_test.shape[0],28,28,1)
    print(X_train.shape)
    print(X_test.shape)
    print(y_train.shape)
    print(y_test.shape)
    
    return X_train, y_train, X_test, y_test

# Finding Optimal Hyperparameters using Hyperas and Hyperopt

In [10]:
def train_cnn_model(X_train, y_train, X_test, y_test):
    # CNN model
    model = Sequential()
    model.add(Conv2D({{choice([32, 64, 128])}}, (3, 3), padding='same', activation='relu', input_shape=(28,28,1)))
    model.add(Conv2D({{choice([32, 64, 128])}}, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout({{uniform(0, 1)}}))

    model.add(Conv2D({{choice([64, 128, 256])}}, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D({{choice([64, 128, 256])}}, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout({{uniform(0, 1)}}))

    model.add(Conv2D({{choice([64, 128, 256])}}, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D({{choice([64, 128, 256])}}, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout({{uniform(0, 1)}}))

    model.add(Flatten())
    model.add(Dense({{choice([128, 256, 512, 1024])}}))
    model.add(Activation({{choice(['relu', 'sigmoid'])}}))
    model.add(Dropout({{uniform(0, 1)}}))
    model.add(Dense(36, activation='softmax'))
    
    model.compile(loss={{choice(['categorical_crossentropy'])}}, metrics=['accuracy'], optimizer={{choice(['rmsprop', 'adam', 'sgd'])}})
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs={{choice([20, 50, 100])}}, batch_size={{choice([16, 32, 64, 128])}}, verbose=2)
    
    score, acc = model.evaluate(X_test, y_test, verbose=2)
    print('Test accuracy: {}'.format(acc))
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

In [11]:
%%time
best_run, best_model = optim.minimize(model=train_cnn_model,
                                          data=get_train_test_data,
                                          algo=tpe.suggest,
                                          max_evals=20,
                                          trials=Trials(),
                                          notebook_name='training_pipeline_research',)
X_train, y_train, X_test, y_test = get_train_test_data()
print("Evaluating best performing model...")
print(best_model.evaluate(X_test, y_test))
print("***********************************************")
print("Best performing model hyper-parameters: ")
print(best_run)
print("***********************************************")

>>> Imports:
#coding=utf-8

try:
    import pandas as pd
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    import cv2
except:
    pass

try:
    import os
except:
    pass

try:
    import pickle
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.preprocessing import OneHotEncoder
except:
    pass

try:
    from keras.models import Sequential
except:
    pass

try:
    from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, Activation
except:
    pass

try:
    from matplotlib import pyplot as plt
except:
    pass

try:
    from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from keras.models import Sequential
except:
    pass

try:
    from keras.layers import Dense, Dropout
except:
    pass

try:
    from keras.optimizers import Adam, SGD
except:
    pass

>>> Hyperas search space:

def get_space(

Epoch 17/50                                           
1829/1829 - 75s - val_accuracy: 0.9492 - accuracy: 0.8386 - loss: 0.5488 - val_loss: 0.1879

Epoch 18/50                                           
1829/1829 - 75s - val_accuracy: 0.9460 - accuracy: 0.8437 - loss: 0.5420 - val_loss: 0.1862

Epoch 19/50                                           
1829/1829 - 74s - val_accuracy: 0.9524 - accuracy: 0.8467 - loss: 0.5319 - val_loss: 0.1734

Epoch 20/50                                           
1829/1829 - 72s - val_accuracy: 0.9534 - accuracy: 0.8532 - loss: 0.5122 - val_loss: 0.1794

Epoch 21/50                                           
1829/1829 - 71s - val_accuracy: 0.9545 - accuracy: 0.8558 - loss: 0.4961 - val_loss: 0.1770

Epoch 22/50                                           
1829/1829 - 69s - val_accuracy: 0.9593 - accuracy: 0.8615 - loss: 0.4821 - val_loss: 0.1588

Epoch 23/50                                           
1829/1829 - 69s - val_accuracy: 0.9574 - accuracy: 0.8592

915/915 - 38s - val_accuracy: 0.8717 - accuracy: 0.7909 - loss: 0.8015 - val_loss: 0.6173

Epoch 18/100                                                                            
915/915 - 39s - val_accuracy: 0.9120 - accuracy: 0.7817 - loss: 0.8325 - val_loss: 0.3520

Epoch 19/100                                                                            
915/915 - 38s - val_accuracy: 0.8958 - accuracy: 0.7828 - loss: 0.8576 - val_loss: 0.4037

Epoch 20/100                                                                            
915/915 - 38s - val_accuracy: 0.8771 - accuracy: 0.7787 - loss: 0.8917 - val_loss: 0.4929

Epoch 21/100                                                                            
915/915 - 38s - val_accuracy: 0.8987 - accuracy: 0.7758 - loss: 0.8709 - val_loss: 0.4029

Epoch 22/100                                                                            
915/915 - 39s - val_accuracy: 0.8952 - accuracy: 0.7771 - loss: 0.8713 - val_loss: 0.3707

Epoch 23/

Epoch 63/100                                                                            
915/915 - 39s - val_accuracy: 0.7924 - accuracy: 0.5493 - loss: 1.9117 - val_loss: 0.8633

Epoch 64/100                                                                            
915/915 - 39s - val_accuracy: 0.7887 - accuracy: 0.5504 - loss: 1.9821 - val_loss: 0.8792

Epoch 65/100                                                                            
915/915 - 40s - val_accuracy: 0.7563 - accuracy: 0.5558 - loss: 1.9102 - val_loss: 1.2040

Epoch 66/100                                                                            
915/915 - 40s - val_accuracy: 0.6990 - accuracy: 0.4906 - loss: 2.2140 - val_loss: 1.2663

Epoch 67/100                                                                            
915/915 - 40s - val_accuracy: 0.7496 - accuracy: 0.5285 - loss: 1.9759 - val_loss: 1.0217

Epoch 68/100                                                                            
915/915 - 4

458/458 - 59s - val_accuracy: 0.0264 - accuracy: 0.0275 - loss: 3.8096 - val_loss: 3.6508

Epoch 8/100                                                                             
458/458 - 60s - val_accuracy: 0.0320 - accuracy: 0.0297 - loss: 3.8065 - val_loss: 3.6169

Epoch 9/100                                                                             
458/458 - 60s - val_accuracy: 0.0294 - accuracy: 0.0273 - loss: 3.8208 - val_loss: 3.6251

Epoch 10/100                                                                            
458/458 - 60s - val_accuracy: 0.0254 - accuracy: 0.0283 - loss: 3.8194 - val_loss: 3.6295

Epoch 11/100                                                                            
458/458 - 59s - val_accuracy: 0.0283 - accuracy: 0.0281 - loss: 3.8174 - val_loss: 3.6392

Epoch 12/100                                                                            
458/458 - 62s - val_accuracy: 0.0257 - accuracy: 0.0286 - loss: 3.8122 - val_loss: 3.6274

Epoch 13/

...

915/915 - 86s - val_accuracy: 0.0264 - accuracy: 0.0289 - loss: 3.6165 - val_loss: 3.5845

Epoch 17/50                                                                              
915/915 - 87s - val_accuracy: 0.0488 - accuracy: 0.0282 - loss: 3.6176 - val_loss: 3.5883

Epoch 18/50                                                                              
915/915 - 88s - val_accuracy: 0.0469 - accuracy: 0.0310 - loss: 3.6108 - val_loss: 3.5766

Epoch 19/50                                                                              
915/915 - 87s - val_accuracy: 0.0347 - accuracy: 0.0306 - loss: 3.6130 - val_loss: 3.5816

Epoch 20/50                                                                              
915/915 - 88s - val_accuracy: 0.0400 - accuracy: 0.0293 - loss: 3.6145 - val_loss: 3.5808

Epoch 21/50                                                                              
915/915 - 90s - val_accuracy: 0.0499 - accuracy: 0.0317 - loss: 3.6090 - val_loss: 3.5731

Epoc

915/915 - 62s - val_accuracy: 0.8932 - accuracy: 0.8361 - loss: 0.6383 - val_loss: 0.4097

Epoch 11/20                                                                              
915/915 - 63s - val_accuracy: 0.8976 - accuracy: 0.8352 - loss: 0.6388 - val_loss: 0.4719

Epoch 12/20                                                                              
915/915 - 63s - val_accuracy: 0.9094 - accuracy: 0.8367 - loss: 0.6474 - val_loss: 0.3600

Epoch 13/20                                                                              
915/915 - 63s - val_accuracy: 0.9047 - accuracy: 0.8360 - loss: 0.6363 - val_loss: 0.3624

Epoch 14/20                                                                              
915/915 - 63s - val_accuracy: 0.9005 - accuracy: 0.8302 - loss: 0.8552 - val_loss: 0.4145

Epoch 15/20                                                                              
915/915 - 62s - val_accuracy: 0.9053 - accuracy: 0.8272 - loss: 0.6874 - val_loss: 0.3584

Epoc

1829/1829 - 157s - val_accuracy: 0.0466 - accuracy: 0.0299 - loss: 3.7745 - val_loss: 3.7323

Epoch 14/100                                                                           
1829/1829 - 156s - val_accuracy: 0.0297 - accuracy: 0.0310 - loss: 3.7685 - val_loss: 3.7225

Epoch 15/100                                                                           
1829/1829 - 157s - val_accuracy: 0.0515 - accuracy: 0.0339 - loss: 3.7525 - val_loss: 3.6980

Epoch 16/100                                                                           
1829/1829 - 157s - val_accuracy: 0.0595 - accuracy: 0.0333 - loss: 3.7487 - val_loss: 3.6401

Epoch 17/100                                                                           
1829/1829 - 157s - val_accuracy: 0.0335 - accuracy: 0.0349 - loss: 3.7503 - val_loss: 3.6647

Epoch 18/100                                                                           
1829/1829 - 156s - val_accuracy: 0.0346 - accuracy: 0.0342 - loss: 3.7485 - val_loss: 3.59

1829/1829 - 157s - val_accuracy: 0.0301 - accuracy: 0.0330 - loss: 3.7545 - val_loss: 3.6680

Epoch 59/100                                                                           
1829/1829 - 156s - val_accuracy: 0.0477 - accuracy: 0.0286 - loss: 3.7864 - val_loss: 3.6682

Epoch 60/100                                                                           
1829/1829 - 156s - val_accuracy: 0.0291 - accuracy: 0.0312 - loss: 3.7750 - val_loss: 3.7726

Epoch 61/100                                                                           
1829/1829 - 156s - val_accuracy: 0.0269 - accuracy: 0.0298 - loss: 3.7754 - val_loss: 3.7229

Epoch 62/100                                                                           
1829/1829 - 156s - val_accuracy: 0.0264 - accuracy: 0.0322 - loss: 3.7695 - val_loss: 3.7248

Epoch 63/100                                                                           
1829/1829 - 156s - val_accuracy: 0.0396 - accuracy: 0.0348 - loss: 3.7613 - val_loss: 3.67

229/229 [==============================] - 4s 16ms/step - accuracy: 0.9683 - loss: 0.1265
[0.12646032869815826, 0.9682886600494385]
***********************************************
Best performing model hyper-parameters: 
{'batch_size_1': 0, 'batch_size': 0, 'loss': 0, 'Activation': 1, 'optimizer': 2, 'Dropout': 0.11245974166556161, 'Dropout_2': 0.7342146978592597, 'Conv2D_3': 0, 'Conv2D_2': 2, 'Dropout_1': 0.3207527760045966, 'Conv2D_1': 0, 'Conv2D': 2, 'Dropout_3': 0.692539034315719, 'Conv2D_4': 1, 'Dense': 1, 'epochs': 1, 'Conv2D_5': 1, 'epochs_1': 0}
***********************************************
CPU times: user 26d 10h 23min 13s, sys: 17h 44min 25s, total: 27d 4h 7min 38s
Wall time: 1d 6h 59min 39s
